In [73]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.interpolate import interp1d
from scipy.integrate import *
from information_scores import *
from sklearn.linear_model import Lasso
import pandas as pd

rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [8, 16]

In [68]:
# Layout the lynx and hare data

time = np.arange(0,1905-1845,2) # original time 

hare = (np.array([20,20,52,83,64,68,83,12,3,150,110,
                  60,7,10,70,100,92,70,10,11,137,137,
                  18,22,52,83,18,10,9,65])) # original hare population values


lynx = (np.array([32,50,12,10,13,36,15,12,6,6,65,70,
                  40,9,20,34,45,40,15,15,60,80,26,18,
                  37,50,35,12,12,25])) # original lynx population values

time_new = np.linspace(0,1903-1845,500) # new time value
dt = np.abs(time_new[1]-time_new[2]) # the new time step

f_hare = interp1d(time,hare,'cubic') # hare inteprolation function

hare_interp = f_hare(time_new) # inteprolated hare population values

f_lynx = interp1d(time,lynx,'cubic') # lynx interpolation function

lynx_interp = f_lynx(time_new) # interpolated lynx population values

X = np.vstack([hare_interp,lynx_interp]) # data matrix

# create the X and Xprime matrices to pass to the SINDy function

# Fourth Order Central Difference
X1 = X[0,:]
X2 = X[1,:]

dX1 = (1/(12*dt)) * (-X1[4:] + 8*X1[3:-1] - 8*X1[1:-3] + X1[:-4])
X1 = X1[2:-2]

dX2 = (1/(12*dt)) * (-X2[4:] + 8*X2[3:-1] - 8*X2[1:-3] + X2[:-4])
X2 = X2[2:-2]

dX = np.vstack([dX1,dX2]) # data matrix

# Trim first and last two that are lost in derivative
X = X[:,2:-2].T
Xprime = dX.T


print(X.shape,Xprime.shape) # print out the shapes of the data matrices as a sanity check

(496, 2) (496, 2)


In [89]:
# SINDy Function Definitions

def build_library(X):
    
    n = X.shape[0]
    Theta = np.zeros((n,1))
    function_label = []
    
    # first order polynomial in x (first state)
    first_state = X[:,0]
    Theta[:,0] = first_state
    function_label.append('x')
    
    # first order polynomial in y (second state)
    second_state = np.array([X[:,1]]).T
    Theta = np.append(Theta,second_state,axis=1)
    function_label.append('y')
    
    # first order polynomial in xy (product of states)
    product = np.array([X[:,0]*X[:,1]]).T
    Theta = np.append(Theta,product,axis = 1)
    function_label.append('xy')
    
    # second order polynomial in x
    x2 = np.array([X[:,0]*X[:,0]]).T
    Theta = np.append(Theta,x2,axis = 1)
    function_label.append('x^2')
    
    # second order polynomial in y
    y2 = np.array([X[:,1]*X[:,1]]).T
    Theta = np.append(Theta,y2,axis = 1)
    function_label.append('y^2')
    
    # second order polynomial in xy
    xy2 = np.array([(X[:,0]*X[:,1])**2]).T
    Theta = np.append(Theta,xy2,axis = 1)
    function_label.append('(xy)^2')
    
    period1 = 5 
    # sin(omega*x), omega = (2*pi)/T , T = 5 years
    sinx = np.array([np.sin(2*np.pi*X[:,0]/period1)]).T
    Theta = np.append(Theta,sinx,axis = 1)
    function_label.append('sin(omega1*x)')
    
    # sin(omega*y), omega = (2*pi)/T , T = 5 years
    siny = np.array([np.sin(2*np.pi*X[:,1]/period1)]).T
    Theta = np.append(Theta,siny,axis = 1)
    function_label.append('sin(omega1*y)')
    
    # sin(omega*xy), omega = (2*pi)/T , T = 5 years
    sinxy = np.array([np.sin(2*np.pi*(X[:,1]*X[:,0])/period1)]).T
    Theta = np.append(Theta,sinxy,axis = 1)
    function_label.append('sin(omega1*xy)')
    
    period2 = 15
    # sin(omega*x), omega = (2*pi)/T , T = 15 years
    sinx2 = np.array([np.sin(2*np.pi*X[:,0]/period2)]).T
    Theta = np.append(Theta,sinx2,axis = 1)
    function_label.append('sin(omega2*x)')
    
    # sin(omega*y), omega = (2*pi)/T , T = 15 years
    siny2 = np.array([np.sin(2*np.pi*X[:,1]/period2)]).T
    Theta = np.append(Theta,siny2,axis = 1)
    function_label.append('sin(omega2*y)')
    
    # sin(omega*xy), omega = (2*pi)/T , T = 15 years
    sinxy2 = np.array([np.sin(2*np.pi*(X[:,1]*X[:,0])/period2)]).T
    Theta = np.append(Theta,sinxy2,axis = 1)
    function_label.append('sin(omega2*xy)')
    
    return Theta, function_label

# def Sparse_Regression(Theta,dXdt,thresh):
    
#     Xi = np.linalg.lstsq(Theta,dXdt,rcond=None)[0] # Initial guess: Least-squares
#     smallinds = np.where(np.abs(Xi) < thresh) # find where the entries is Xi are below the threshold
#     Xi[smallinds] = 0 # sparsify!!!!! Hell yeah!!!
    
#     return Xi

def Sparse_Regression(Theta,dXdt,thresh):
    
    lassoreg = Lasso(alpha=thresh,normalize=True, max_iter=1e5) # sparsify!!!!! Hell yeah!!!
    lassoreg.fit(Theta,dXdt)
    Xi = lassoreg.coef_
    
    return Xi.T

In [90]:
Theta,function_label = build_library(X) # build the lbrary of functions (Theta)
threshold = 0.001 # thresholding paramater for sparsification
Xi = Sparse_Regression(Theta,Xprime,threshold) # obtain Xi ("ksi") that gives us the coefficients
Xi = np.asarray(Xi)
print(Xi) # print out Xi as a sanity check

IndexError: too many indices for array

In [91]:
# define a sample ODE to test RK4 on

def Lotka_Volterra(t,X,blah):
    
    x = X[0]
    y = X[1]
    
    omega1 = 2*np.pi/5
    omega2 = 2*np.pi/15
    
    xdot = (0.47064556*x+0.07136272*y-0.01110213*x*y-0.00118093*x**2-0.0050453*y**2+6.53842294*np.sin(omega2*y)
             +1.1774889*np.sin(omega1*x)-2.26756052*np.sin(omega1*y))

    
    ydot = (-0.15382964*x - 0.1406684*y+5.63696948e-03*x*y+0.00179203*x**2-0.00275862*y**2+
            1.25269343*np.sin(omega2*x))

    return np.asarray([xdot,ydot])

In [70]:
# initial and final times
t_start = time_new[0]; t_stop = time_new[-1]

h = 0.001
n = int((t_stop-t_start)/h) # number of iterations

P0 = np.asarray([20.0, 32.0])
X = P0
X_sol = X0
t_sol = np.asarray([0])

# RK4
for i in range(0, n):
    # obtain k1 value
    k1 = h*func(X,t)
    # obtain k2 value
    k2 = h*func(X+k1/2,t+h/2)
    # obtain k3 value
    k3 = h*func(X+k2/2,t+h/2)
    # obtain k4 value
    k4 = h*func(X+k3,t+h)
    # obtain next time step in state variables
    X += (k1+2*k2+2*k3+k4)/6
    # update independent variable using the step size
    t += h
    # collect values
    X_sol = np.vstack([X_sol,X])
    t_sol = np.vstack([t_sol,t])

In [92]:
P0 = [20, 32]
tspan = np.asarray([time_new[0],time_new[-1]])

Ps = solve_ivp(Lotka_Volterra, tspan, P0, method = 'LSODA', args = (1))
hare_pop = Ps.y[0,:]
lynx_pop = Ps.y[1,:]

plt.plot(hare_pop)
plt.show()
plt.plot(lynx_pop)
plt.show()

/usr/local/lib/python3.5/dist-packages/scipy/integrate/_ivp/common.py:41: UserWarning: The following arguments have no effect for a chosen solver: `args`.
  .format(", ".join("`{}`".format(x) for x in extraneous)))


TypeError: Lotka_Volterra() missing 1 required positional argument: 'blah'

In [86]:
val = Ps.y
val.shape

(2, 613)

## Fourth Order Runge-Kutta

In [6]:
def RK4(func, yinit, xspan, h):
    m = len(yinit)
    n = int((xspan[-1] - xspan[0]) / h)

    x = xspan[0]
    y = yinit

    xsol = np.empty((0))
    xsol = np.append(xsol, x)

    ysol = np.empty((0))
    ysol = np.append(ysol, y)

    for i in range(n):
        k1 = func(x,y)

        yp2 = y + k1*(h/2)

        k2 = func(x+h/2, yp2)

        yp3 = y + k2*(h/2)

        k3 = func(x+h/2, yp3)

        yp4 = y + k3*h

        k4 = func(x+h, yp4)

        for j in range(m):
            y[j] = y[j] + (h/6)*(k1[j] + 2*k2[j] + 2*k3[j] + k4[j])

        x = x + h
        xsol = np.append(xsol, x)

        for r in range(len(y)):
            ysol = np.append(ysol, y[r])  # Save all new y's

    return [xsol, ysol]

In [8]:
P0 = [20, 32]
h = np.abs(time_new[0]-time_new[1])
tstart = time_new[0]; tstop = time_new[-1]
tspan = np.asarray([tstart, tstop])
Ps = RK4(Lotka_Volterra, P0, time_new, h)
hare_pop = Ps[:,0]
lynx_pop = Ps[:,1]

MemoryError: 

In [7]:
print(time_new[0])

0.0


$ \dot{u} = 4x+10 \sin x - y $ 

$ \dot{y} = u $

In [16]:
def func(X,t):
    
    u = X[0]; y = X[1]
    
    du = 4*t+10*np.sin(t)-y
    dy = u
    
    return np.asarray([du,dy])

In [28]:
from math import sin, cos, pi
# the analytical solution
f = lambda x: 9*pi*cos(x) + 7*sin(x) + 4*x - 5*x*cos(x)
df = lambda x: -9*pi*sin(x) + 7*cos(x) + 4 - 5*(cos(x) - x*sin(x))
# initial values
t = pi # initial time
t_end = 2*pi # final time
X0 = np.asarray([2,0.0]) # initial condition
h = 0.1 # step size
n = int((t_end-t)/h) # number of iterations

X = X0
X_sol = X0


# RK4
for i in range(1, n+1):
    # obtain k1 value
    k1 = h*func(X,t)
    # obtain k2 value
    k2 = h*func(X+k1/2,t+h/2)
    # obtain k3 value
    k3 = h*func(X+k2/2,t+h/2)
    # obtain k4 value
    k4 = h*func(X+k3,t+h)
    # obtain next time step in state variables
    X += (k1+2*k2+2*k3+k4)/6
    # update independent variable using the step size
    t += h
    
    # collect values
    X_sol = np.vstack([X_sol,X])
    
    
    

((32, 2), 31)